# Fraud Detection Demo

This notebook generates a synthetic event stream, computes rolling-window features, trains an Isolation Forest baseline, and produces anomaly scores with a simple thresholding policy.

In [ ]:
import pandas as pd
from src.common.synthetic_data import generate_synthetic_fraud_events
from src.fraud.model import train_isolation_forest, score

# Generate synthetic events
 df = generate_synthetic_fraud_events(n=8000)
 df.head()

In [ ]:
import numpy as np

# Minimal feature set for demo
X = pd.DataFrame({
    'latency_ms': df['latency_ms'],
    'is_error': (df['status_code']>=400).astype(int)
})

model = train_isolation_forest(X)

df['anomaly_score'] = score(model, X)

df[['timestamp','event_type','latency_ms','status_code','anomaly_score']].head()

In [ ]:
# Dynamic threshold example: flag top 1% most anomalous events
threshold = df['anomaly_score'].quantile(0.99)
df['risk'] = np.where(df['anomaly_score'] >= threshold, 'HIGH', 'NORMAL')
df['risk'].value_counts()

## Notes
- This is a baseline demo. In the full architecture, features include velocity windows, entropy, and deviation scores.
- Data is synthetic only.